# Twitter Data for the Project

In this python file we are going to use tweepy to get the tweets that contain the keywords: "Nintendo", "PlayStation", "Xbox" and "Videogames".

The first part is for obtaining the tweets, and saving them in a file.

The second part is for reading from the file all the tweets saved to insert them in MongoDB.

The last part is related to analysis.

In [9]:
import tweepy

# Variables that contains the user credentials to access Twitter API
access_key = "766667542240882688-NtPIOdl9CCKLZ7UBn8JaLIiJ6k3wYQN"
access_secret = "RYWpbnHH3cbh7R7MXs8qVzgGwTBAcPn79IYeXISGvEiHO"
consumer_key = "GU8slDLVo4HygqL15RCVfQWd0"
consumer_secret = "Tlp6rArgxhGrySTrbSfgKgCyA2UUf4x41UEDYu3OaOG9ZEccGY"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

filename = "tweets_videogames.data"

class CustomStreamListener(tweepy.StreamListener):
    def __init__(self, api):
        self.api = api
        super(tweepy.StreamListener, self).__init__()

    def on_data(self, tweet):
        #print tweet
        try:
            with open(filename, 'a') as _file:
                _file.write(tweet)
        except:
            pass

    def on_error(self, status_code):
        return True  # Don't kill the stream

    def on_timeout(self):
        return True  # Don't kill the stream

sapi = tweepy.streaming.Stream(auth, CustomStreamListener(api))
sapi.filter(track=["Nintendo", "PlayStation", "Xbox", "Videogames"])


ProtocolError: ('Connection broken: IncompleteRead(0 bytes read, 499 more expected)', IncompleteRead(0 bytes read, 499 more expected))

This is just for the purpose to know how many tweets there are in the file.

In [10]:
amount_tweets = 0
filename = "tweets_videogames.data"
_file = open(filename, 'r')
for line in _file.readlines():
    amount_tweets += 1
_file.close()
print amount_tweets

115416


This next part is for loading the tweets downloaded in mongo.

In [ ]:
import json
import pymongo

filename = "tweets_videogames.data"
db = pymongo.MongoClient().project
_file = open(filename, 'r')
for line in _file.readlines():
    #print line
    db.tweets.insert(json.loads(line))
_file.close()

And this last part is to analyze the data in mongo.

In [2]:
print "Nothing done here yet."

Nothing done here yet.
